In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
import xgboost as xgb
import requests
import json
import io
import logging
import csv
import datetime
from backend.utils.data_loader import DataLoader

In [2]:
data_loader = DataLoader()
data_loader.generate_raw_data()

INFO:log:grabbing data for Blackhawks
INFO:log:grabbing data for MapleLeafs
INFO:log:grabbing data for Canadiens
INFO:log:grabbing data for Bruins
INFO:log:grabbing data for Sabres
INFO:log:grabbing data for Senators
INFO:log:grabbing data for RedWings
INFO:log:grabbing data for Lightning
INFO:log:grabbing data for Panthers
INFO:log:grabbing data for Flyers
INFO:log:grabbing data for Rangers
INFO:log:grabbing data for Islanders
INFO:log:grabbing data for Hurricane
INFO:log:grabbing data for Penguins
INFO:log:grabbing data for BlueJackets
INFO:log:grabbing data for Devils
INFO:log:grabbing data for Capitals
INFO:log:grabbing data for Jets
INFO:log:grabbing data for Wild
INFO:log:grabbing data for Predators
INFO:log:grabbing data for Blues
INFO:log:grabbing data for Stars
INFO:log:grabbing data for Avalanche
INFO:log:grabbing data for Coyotes
INFO:log:grabbing data for Flames
INFO:log:grabbing data for Sharks
INFO:log:grabbing data for Kings
INFO:log:grabbing data for GoldenKnights
INFO:

In [46]:
df = pd.read_csv('raw-training-data.csv')

In [47]:
df_2021_and_later = df[df.season.astype(np.int32) >= 2021]

In [39]:
df_2021_and_later

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,...,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst
4969,CHI,2021,CHI,2021020005,CHI,COL,AWAY,20211013,Team Level,other,...,6.000,4.0,4.000,1.0,0.037,0.109,0.109,0.114,0.114,0.113
4970,CHI,2021,CHI,2021020005,CHI,COL,AWAY,20211013,Team Level,all,...,74.724,49.0,50.600,7.0,0.593,0.441,0.441,3.714,3.763,3.667
4971,CHI,2021,CHI,2021020005,CHI,COL,AWAY,20211013,Team Level,5on5,...,58.724,37.0,38.600,4.0,0.442,0.087,0.087,3.054,3.103,3.022
4972,CHI,2021,CHI,2021020005,CHI,COL,AWAY,20211013,Team Level,4on5,...,9.000,7.0,7.000,0.0,0.109,0.245,0.245,0.526,0.526,0.513
4973,CHI,2021,CHI,2021020005,CHI,COL,AWAY,20211013,Team Level,5on4,...,1.000,1.0,1.000,2.0,0.006,0.000,0.000,0.020,0.020,0.020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183684,ANA,2023,ANA,2023021311,ANA,VGK,AWAY,20240418,Team Level,other,...,6.000,6.0,6.000,0.0,0.131,0.980,0.980,0.598,0.598,0.409
183685,ANA,2023,ANA,2023021311,ANA,VGK,AWAY,20240418,Team Level,all,...,63.741,45.0,42.965,4.0,0.466,1.222,1.222,2.685,2.581,2.381
183686,ANA,2023,ANA,2023021311,ANA,VGK,AWAY,20240418,Team Level,5on5,...,50.741,35.0,32.965,4.0,0.279,0.154,0.154,1.788,1.684,1.674
183687,ANA,2023,ANA,2023021311,ANA,VGK,AWAY,20240418,Team Level,4on5,...,7.000,4.0,4.000,0.0,0.057,0.087,0.087,0.299,0.299,0.298


In [48]:
join_cols = ['season', 'gameId']

In [49]:
df_2021_and_later = df_2021_and_later[df_2021_and_later.situation == 'all']
df_2021_and_later_home = df_2021_and_later[df_2021_and_later.home_or_away == 'HOME']
df_2021_and_later_away = df_2021_and_later[df_2021_and_later.home_or_away == 'AWAY']

In [50]:
df_2021_and_later = pd.merge(right=df_2021_and_later_away, left=df_2021_and_later_home, on=join_cols, how='inner', suffixes=['_home', '_away'])

In [52]:
df_2021_and_later.columns.to_list()

['team_home',
 'season',
 'name_home',
 'gameId',
 'playerTeam_home',
 'opposingTeam_home',
 'home_or_away_home',
 'gameDate_home',
 'position_home',
 'situation_home',
 'xGoalsPercentage_home',
 'corsiPercentage_home',
 'fenwickPercentage_home',
 'iceTime_home',
 'xOnGoalFor_home',
 'xGoalsFor_home',
 'xReboundsFor_home',
 'xFreezeFor_home',
 'xPlayStoppedFor_home',
 'xPlayContinuedInZoneFor_home',
 'xPlayContinuedOutsideZoneFor_home',
 'flurryAdjustedxGoalsFor_home',
 'scoreVenueAdjustedxGoalsFor_home',
 'flurryScoreVenueAdjustedxGoalsFor_home',
 'shotsOnGoalFor_home',
 'missedShotsFor_home',
 'blockedShotAttemptsFor_home',
 'shotAttemptsFor_home',
 'goalsFor_home',
 'reboundsFor_home',
 'reboundGoalsFor_home',
 'freezeFor_home',
 'playStoppedFor_home',
 'playContinuedInZoneFor_home',
 'playContinuedOutsideZoneFor_home',
 'savedShotsOnGoalFor_home',
 'savedUnblockedShotAttemptsFor_home',
 'penaltiesFor_home',
 'penalityMinutesFor_home',
 'faceOffsWonFor_home',
 'hitsFor_home',
 'take

In [53]:
disregard = ['team_home',
 'season',
 'name_home',
 'gameId',
 'playerTeam_home',
 'opposingTeam_home',
 'home_or_away_home',
 'gameDate_home',
 'position_home',
 'situation_home',
 'team_away',
 'season',
 'name_away',
 'gameId',
 'playerTeam_away',
 'opposingTeam_away',
 'home_or_away_away',
 'gameDate_away',
 'position_away',
 'situation_away',
 
 ]

In [56]:
cols = list(set(df_2021_and_later.columns.to_list()) - set(disregard))
rel_cols = [col for col in cols if col.endswith('_home') or col.endswith("_away")]
home_rel_cols = [col for col in rel_cols if col.endswith('_home')]
away_rel_cols = [col for col in rel_cols if col.endswith('_away')]

In [57]:
home_rel_cols

['iceTime_home',
 'xFreezeAgainst_home',
 'scoreVenueAdjustedxGoalsAgainst_home',
 'flurryAdjustedxGoalsFor_home',
 'hitsFor_home',
 'totalShotCreditAgainst_home',
 'savedShotsOnGoalFor_home',
 'xPlayStoppedAgainst_home',
 'lowDangerShotsAgainst_home',
 'playContinuedOutsideZoneAgainst_home',
 'mediumDangerShotsFor_home',
 'xReboundsFor_home',
 'xFreezeFor_home',
 'reboundGoalsFor_home',
 'unblockedShotAttemptsAgainst_home',
 'savedUnblockedShotAttemptsFor_home',
 'xGoalsFromActualReboundsOfShotsAgainst_home',
 'scoreFlurryAdjustedTotalShotCreditFor_home',
 'lowDangerGoalsAgainst_home',
 'fenwickPercentage_home',
 'xPlayContinuedOutsideZoneAgainst_home',
 'xGoalsFromActualReboundsOfShotsFor_home',
 'scoreAdjustedShotsAttemptsFor_home',
 'highDangerGoalsAgainst_home',
 'penaltiesAgainst_home',
 'scoreAdjustedTotalShotCreditAgainst_home',
 'blockedShotAttemptsFor_home',
 'shotsOnGoalAgainst_home',
 'giveawaysFor_home',
 'goalsFor_home',
 'faceOffsWonFor_home',
 'reboundsFor_home',
 'lowD

In [65]:
col_map = {}
home_rel_cols.sort()
away_rel_cols.sort()
for home, away in zip(home_rel_cols, away_rel_cols):
    col_map.update({home: away})

In [72]:
col_map

{'blockedShotAttemptsAgainst_home': 'blockedShotAttemptsAgainst_away',
 'blockedShotAttemptsFor_home': 'blockedShotAttemptsFor_away',
 'corsiPercentage_home': 'corsiPercentage_away',
 'dZoneGiveawaysAgainst_home': 'dZoneGiveawaysAgainst_away',
 'dZoneGiveawaysFor_home': 'dZoneGiveawaysFor_away',
 'faceOffsWonAgainst_home': 'faceOffsWonAgainst_away',
 'faceOffsWonFor_home': 'faceOffsWonFor_away',
 'fenwickPercentage_home': 'fenwickPercentage_away',
 'flurryAdjustedxGoalsAgainst_home': 'flurryAdjustedxGoalsAgainst_away',
 'flurryAdjustedxGoalsFor_home': 'flurryAdjustedxGoalsFor_away',
 'flurryScoreVenueAdjustedxGoalsAgainst_home': 'flurryScoreVenueAdjustedxGoalsAgainst_away',
 'flurryScoreVenueAdjustedxGoalsFor_home': 'flurryScoreVenueAdjustedxGoalsFor_away',
 'freezeAgainst_home': 'freezeAgainst_away',
 'freezeFor_home': 'freezeFor_away',
 'giveawaysAgainst_home': 'giveawaysAgainst_away',
 'giveawaysFor_home': 'giveawaysFor_away',
 'goalsAgainst_home': 'goalsAgainst_away',
 'goalsFor_ho

In [67]:
root = ["_".join(col.split("_")[:-1]) for col in home_rel_cols]

In [68]:
root

['blockedShotAttemptsAgainst',
 'blockedShotAttemptsFor',
 'corsiPercentage',
 'dZoneGiveawaysAgainst',
 'dZoneGiveawaysFor',
 'faceOffsWonAgainst',
 'faceOffsWonFor',
 'fenwickPercentage',
 'flurryAdjustedxGoalsAgainst',
 'flurryAdjustedxGoalsFor',
 'flurryScoreVenueAdjustedxGoalsAgainst',
 'flurryScoreVenueAdjustedxGoalsFor',
 'freezeAgainst',
 'freezeFor',
 'giveawaysAgainst',
 'giveawaysFor',
 'goalsAgainst',
 'goalsFor',
 'highDangerGoalsAgainst',
 'highDangerGoalsFor',
 'highDangerShotsAgainst',
 'highDangerShotsFor',
 'highDangerxGoalsAgainst',
 'highDangerxGoalsFor',
 'hitsAgainst',
 'hitsFor',
 'iceTime',
 'lowDangerGoalsAgainst',
 'lowDangerGoalsFor',
 'lowDangerShotsAgainst',
 'lowDangerShotsFor',
 'lowDangerxGoalsAgainst',
 'lowDangerxGoalsFor',
 'mediumDangerGoalsAgainst',
 'mediumDangerGoalsFor',
 'mediumDangerShotsAgainst',
 'mediumDangerShotsFor',
 'mediumDangerxGoalsAgainst',
 'mediumDangerxGoalsFor',
 'missedShotsAgainst',
 'missedShotsFor',
 'penalityMinutesAgainst',

In [77]:
i = 0
for home, away in col_map.items():
    df_2021_and_later[f'{root[i]}_diff'] = df_2021_and_later[home] - df_2021_and_later[away]
    i += 1

In [79]:
diff_cols = [col for col in df_2021_and_later.columns.to_list() if col.endswith("diff")]

In [85]:
bad_dates = df_2021_and_later[df_2021_and_later['goalsFor_diff'] == 0].gameDate_home.to_list()
bad_game_ids = df_2021_and_later[df_2021_and_later['goalsFor_diff'] == 0].gameId.to_list()

In [100]:
df_2021_and_later_good = df_2021_and_later[df_2021_and_later.goalsFor_diff != 0]
df_2021_and_later_bad = df_2021_and_later[df_2021_and_later.goalsFor_diff == 0]

In [137]:
df_2021_and_later_bad = df_2021_and_later_bad.drop(labels=['goalsFor_diff'], axis=1)
df_2021_and_later_bad = pd.merge(left=df_2021_and_later_bad, right=df, on='gameId', how='inner')

In [138]:
df_2021_and_later = pd.concat([df_2021_and_later_good, df_2021_and_later_bad], axis=0)

In [141]:
df_2021_and_later['target'] = np.where(df_2021_and_later.goalsFor_diff > 0, 1, 0)

/var/folders/4d/mtn14h1x1kx9qj0_9ng0wzvw0000gn/T/ipykernel_33915/2204221079.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2021_and_later['target'] = np.where(df_2021_and_later.goalsFor_diff > 0, 1, 0)


In [143]:
df_2021_and_later[['target', 'goalsFor_diff']]

,target,goalsFor_diff
0,0,-3.0
1,0,-3.0
2,0,-3.0
3,0,-1.0
4,1,4.0
...,...,...
204,0,-1.0
205,1,1.0
206,1,1.0
207,1,1.0


In [ ]:
diff_cols

In [145]:
df_2021_and_later_rel = df_2021_and_later[['target', *diff_cols]]

In [147]:
df_2021_and_later_rel

,target,iceTime_diff,blockedShotAttemptsAgainst_diff,blockedShotAttemptsFor_diff,corsiPercentage_diff,dZoneGiveawaysAgainst_diff,dZoneGiveawaysFor_diff,faceOffsWonAgainst_diff,faceOffsWonFor_diff,fenwickPercentage_diff,...,xOnGoalAgainst_diff,xOnGoalFor_diff,xPlayContinuedInZoneAgainst_diff,xPlayContinuedInZoneFor_diff,xPlayContinuedOutsideZoneAgainst_diff,xPlayContinuedOutsideZoneFor_diff,xPlayStoppedAgainst_diff,xPlayStoppedFor_diff,xReboundsAgainst_diff,xReboundsFor_diff
0,0,0.0,-7.0,7.0,0.1694,3.0,-3.0,7.0,-7.0,0.1494,...,-7.602,7.602,-5.623,5.623,-3.086,3.086,-0.301,0.301,-0.745,0.745
1,0,0.0,-4.0,4.0,0.0000,-2.0,2.0,6.0,-6.0,-0.0512,...,3.222,-3.222,0.983,-0.983,1.367,-1.367,0.065,-0.065,0.031,-0.031
2,0,0.0,-11.0,11.0,0.1526,1.0,-1.0,-3.0,3.0,0.0770,...,-5.324,5.324,-2.844,2.844,-1.454,1.454,-0.179,0.179,-0.918,0.918
3,0,0.0,17.0,-17.0,-0.2942,-1.0,1.0,9.0,-9.0,-0.2000,...,12.545,-12.545,7.587,-7.587,5.952,-5.952,0.381,-0.381,0.553,-0.553
4,1,0.0,2.0,-2.0,-0.0350,-1.0,1.0,-12.0,12.0,-0.0250,...,-0.001,0.001,1.432,-1.432,0.704,-0.704,0.046,-0.046,0.357,-0.357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0,0.0,-7.0,7.0,0.0240,-1.0,1.0,-6.0,6.0,-0.0416,...,4.593,-4.593,0.513,-0.513,2.720,-2.720,0.002,-0.002,0.066,-0.066
205,1,0.0,2.0,-2.0,-0.1380,2.0,-2.0,2.0,-2.0,-0.1556,...,9.825,-9.825,6.283,-6.283,3.729,-3.729,0.338,-0.338,0.628,-0.628
206,1,0.0,0.0,0.0,-0.2056,0.0,0.0,10.0,-10.0,-0.2816,...,21.655,-21.655,11.799,-11.799,8.213,-8.213,0.691,-0.691,1.613,-1.613
207,1,0.0,9.0,-9.0,-0.0802,-4.0,4.0,-7.0,7.0,-0.0200,...,1.569,-1.569,2.094,-2.094,1.427,-1.427,0.005,-0.005,0.049,-0.049


In [ ]:
correlations_with_target = {}
for col in diff_cols:
    corr = abs(df_2021_and_later_rel['target'].corr(df_2021_and_later_rel[col]))
    correlations_with_target.update({col:corr})
json.dump(correlations_with_target, open('correlations.json', 'x'))

In [163]:
df_diff_target = df_2021_and_later[['target', *correlations_with_target]]

In [172]:
df_diff_target.iloc[:, 85:]

,xGoalsFor_diff,xGoalsFromActualReboundsOfShotsAgainst_diff,xGoalsFromActualReboundsOfShotsFor_diff,xGoalsFromxReboundsOfShotsAgainst_diff,xGoalsFromxReboundsOfShotsFor_diff,xGoalsPercentage_diff,xOnGoalAgainst_diff,xOnGoalFor_diff,xPlayContinuedInZoneAgainst_diff,xPlayContinuedInZoneFor_diff,xPlayContinuedOutsideZoneAgainst_diff,xPlayContinuedOutsideZoneFor_diff,xPlayStoppedAgainst_diff,xPlayStoppedFor_diff,xReboundsAgainst_diff,xReboundsFor_diff
0,1.926,-0.296,0.296,-0.156,0.156,0.5050,-7.602,7.602,-5.623,5.623,-3.086,3.086,-0.301,0.301,-0.745,0.745
1,-0.825,0.987,-0.987,0.051,-0.051,-0.1572,3.222,-3.222,0.983,-0.983,1.367,-1.367,0.065,-0.065,0.031,-0.031
2,1.390,0.260,-0.260,-0.302,0.302,0.1628,-5.324,5.324,-2.844,2.844,-1.454,1.454,-0.179,0.179,-0.918,0.918
3,0.883,-0.374,0.374,0.114,-0.114,0.2156,12.545,-12.545,7.587,-7.587,5.952,-5.952,0.381,-0.381,0.553,-0.553
4,0.257,-0.060,0.060,0.074,-0.074,0.0758,-0.001,0.001,1.432,-1.432,0.704,-0.704,0.046,-0.046,0.357,-0.357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,-0.578,0.196,-0.196,0.014,-0.014,-0.1058,4.593,-4.593,0.513,-0.513,2.720,-2.720,0.002,-0.002,0.066,-0.066
205,-0.478,-0.213,0.213,0.183,-0.183,-0.0994,9.825,-9.825,6.283,-6.283,3.729,-3.729,0.338,-0.338,0.628,-0.628
206,-1.672,-0.050,0.050,0.385,-0.385,-0.2642,21.655,-21.655,11.799,-11.799,8.213,-8.213,0.691,-0.691,1.613,-1.613
207,0.774,-0.958,0.958,0.022,-0.022,0.1126,1.569,-1.569,2.094,-2.094,1.427,-1.427,0.005,-0.005,0.049,-0.049


In [175]:
dups = ['blockedShotAttemptsAgainst_diff', 'dZoneGiveawaysAgainst_diff', 'faceOffsWonAgainst_diff', 'flurryAdjustedxGoalsAgainst_diff', 'flurryScoreVenueAdjustedxGoalsAgainst_diff',
        'freezeAgainst_diff', 'giveawaysAgainst_diff', 'highDangerGoalsAgainst_diff', 'highDangerShotsAgainst_diff', 'highDangerxGoalsAgainst_diff', 'hitsAgainst_diff', 'lowDangerGoalsAgainst_diff',
        'lowDangerShotsAgainst_diff', 'lowDangerxGoalsAgainst_diff', 'mediumDangerGoalsAgainst_diff', 'mediumDangerShotsAgainst_diff', 'mediumDangerxGoalsAgainst_diff', 'missedShotsAgainst_diff',
        'penalityMinutesAgainst_diff', 'penaltiesAgainst_diff', 'playContinuedInZoneAgainst_diff', 'playContinuedOutsideZoneAgainst_diff', 'playStoppedAgainst_diff', 'reboundGoalsAgainst_diff',
        'reboundsAgainst_diff', 'reboundxGoalsAgainst_diff', 'savedShotsOnGoalAgainst_diff', 'savedUnblockedShotAttemptsAgainst_diff', 'scoreAdjustedShotsAttemptsAgainst_diff', 'scoreAdjustedTotalShotCreditAgainst_diff',
        'scoreAdjustedUnblockedShotAttemptsAgainst_diff', 'scoreFlurryAdjustedTotalShotCreditAgainst_diff', 'scoreVenueAdjustedxGoalsAgainst_diff', 'shotAttemptsAgainst_diff', 'shotsOnGoalAgainst_diff',
        'takeawaysAgainst_diff', 'totalShotCreditAgainst_diff', 'unblockedShotAttemptsAgainst_diff', 'xFreezeAgainst_diff', 'xGoalsAgainst_diff', 'xGoalsFromActualReboundsOfShotsAgainst_diff',
        'xGoalsFromxReboundsOfShotsAgainst_diff', 'xOnGoalAgainst_diff', 'xPlayContinuedInZoneAgainst_diff', 'xPlayContinuedOutsideZoneAgainst_diff', 'xPlayStoppedAgainst_diff', 'xReboundsAgainst_diff']

In [176]:
df_diff_target = df_diff_target.drop(labels=dups, axis=1)

In [178]:
df_diff_target = df_diff_target.drop(labels=['iceTime_diff'], axis=1)

In [186]:
df_diff_target = df_diff_target.drop(labels=['goalsFor_diff'], axis=1)

In [191]:
df_diff_target.to_csv("train-set-1.csv", index=False, header=True)

In [189]:
cols = df_diff_target.columns.to_list()

In [192]:
cols

['target',
 'blockedShotAttemptsFor_diff',
 'corsiPercentage_diff',
 'dZoneGiveawaysFor_diff',
 'faceOffsWonFor_diff',
 'fenwickPercentage_diff',
 'flurryAdjustedxGoalsFor_diff',
 'flurryScoreVenueAdjustedxGoalsFor_diff',
 'freezeFor_diff',
 'giveawaysFor_diff',
 'goalsAgainst_diff',
 'highDangerGoalsFor_diff',
 'highDangerShotsFor_diff',
 'highDangerxGoalsFor_diff',
 'hitsFor_diff',
 'lowDangerGoalsFor_diff',
 'lowDangerShotsFor_diff',
 'lowDangerxGoalsFor_diff',
 'mediumDangerGoalsFor_diff',
 'mediumDangerShotsFor_diff',
 'mediumDangerxGoalsFor_diff',
 'missedShotsFor_diff',
 'penalityMinutesFor_diff',
 'penaltiesFor_diff',
 'playContinuedInZoneFor_diff',
 'playContinuedOutsideZoneFor_diff',
 'playStoppedFor_diff',
 'reboundGoalsFor_diff',
 'reboundsFor_diff',
 'reboundxGoalsFor_diff',
 'savedShotsOnGoalFor_diff',
 'savedUnblockedShotAttemptsFor_diff',
 'scoreAdjustedShotsAttemptsFor_diff',
 'scoreAdjustedTotalShotCreditFor_diff',
 'scoreAdjustedUnblockedShotAttemptsFor_diff',
 'scor

In [190]:
correlations_with_target = {}
for col in cols:
    corr = abs(df_diff_target['target'].corr(df_diff_target[col]))
    correlations_with_target.update({col:corr})
json.dump(correlations_with_target, open('correlations.json', 'w'))

In [185]:
correlations_with_target

{'target': 0.9999999999999999,
 'blockedShotAttemptsFor_diff': 0.17756914864731607,
 'corsiPercentage_diff': 0.03029981174674833,
 'dZoneGiveawaysFor_diff': 0.025399091247991515,
 'faceOffsWonFor_diff': 0.07350697250495433,
 'fenwickPercentage_diff': 0.05203597460010471,
 'flurryAdjustedxGoalsFor_diff': 0.40745785220070785,
 'flurryScoreVenueAdjustedxGoalsFor_diff': 0.46260774232542046,
 'freezeFor_diff': 0.10045343882577956,
 'giveawaysFor_diff': 0.049390407005840645,
 'goalsAgainst_diff': 0.8394083377049366,
 'goalsFor_diff': 0.8617684249501953,
 'highDangerGoalsFor_diff': 0.5456682850532285,
 'highDangerShotsFor_diff': 0.2952638085463261,
 'highDangerxGoalsFor_diff': 0.39969858646307554,
 'hitsFor_diff': 0.14724772279996803,
 'lowDangerGoalsFor_diff': 0.45181708395812503,
 'lowDangerShotsFor_diff': 0.0847984842628787,
 'lowDangerxGoalsFor_diff': 0.002283044892423019,
 'mediumDangerGoalsFor_diff': 0.497087084527205,
 'mediumDangerShotsFor_diff': 0.15908553126536673,
 'mediumDangerxGo

In [144]:
df_2021_and_later.to_csv('2021-and-beyond-dataset.csv', index=False, header=True)

In [92]:
f'{str(bad_dates[0])[0:4]}-{str(bad_dates[0])[4:6]}-{str(bad_dates[0])[6:]}'

'2021-11-09'

In [93]:
bad_dates = list(map(lambda x: f'{str(x)[0:4]}-{str(x)[4:6]}-{str(x)[6:]}', bad_dates))

In [ ]:
scores = {}
for date in bad_dates:
    print(f'date: {date}')
    games = requests.get(f"https://api-web.nhle.com/v1/score/{date}").json().get("games")
    for game in games:
        if game.get('id') in bad_game_ids:
            home_score = game.get('homeTeam').get('score')
            away_score = game.get('awayTeam').get('score')
            scores.update({game.get('id'):{"home":home_score, 'away': away_score}})


In [113]:
score_diff = {key: np.array([(scores.get(key).get('home') - scores.get(key).get('away'))]) for key in scores.keys()}

In [125]:
df = pd.DataFrame(data=score_diff)

In [128]:
df = df.transpose().reset_index()

In [134]:
df = df.rename({'index': 'gameId', 0: 'goalsFor_diff'}, axis=1)

In [135]:
df

,gameId,goalsFor_diff
0,2021020188,-1
1,2021020187,1
2,2021020516,-1
3,2021021286,1
4,2022020140,-1
...,...,...
204,2022020867,-1
205,2021020365,-1
206,2021020609,1
207,2022020562,1


In [111]:
score_diff

{2021020188: array([-1]),
 2021020187: array([1]),
 2021020516: array([-1]),
 2021021286: array([1]),
 2022020140: array([-1]),
 2022020321: array([-1]),
 2022020325: array([1]),
 2022020914: array([1]),
 2022020915: array([1]),
 2023020384: array([-1]),
 2021020752: array([1]),
 2022020602: array([-1]),
 2022021076: array([1]),
 2022021077: array([-1]),
 2023020005: array([1]),
 2023020207: array([1]),
 2023020329: array([1]),
 2023020335: array([1]),
 2023020346: array([1]),
 2023020964: array([1]),
 2023021067: array([-1]),
 2023021069: array([1]),
 2021020454: array([1]),
 2021020461: array([-1]),
 2021020933: array([-1]),
 2022020275: array([-1]),
 2022020280: array([1]),
 2022020448: array([1]),
 2022021012: array([-1]),
 2022021018: array([-1]),
 2022021028: array([-1]),
 2023020121: array([1]),
 2023020442: array([-1]),
 2023020610: array([1]),
 2023020608: array([-1]),
 2023021298: array([-1]),
 2021020121: array([1]),
 2021020117: array([1]),
 2022020059: array([1]),
 2022020

In [68]:
def change_string(string):
    return f'{string[0:4]}-{string[4:6]}-{string[6:]}'

In [193]:
df_diff_target

,target,blockedShotAttemptsFor_diff,corsiPercentage_diff,dZoneGiveawaysFor_diff,faceOffsWonFor_diff,fenwickPercentage_diff,flurryAdjustedxGoalsFor_diff,flurryScoreVenueAdjustedxGoalsFor_diff,freezeFor_diff,giveawaysFor_diff,...,xFreezeFor_diff,xGoalsFor_diff,xGoalsFromActualReboundsOfShotsFor_diff,xGoalsFromxReboundsOfShotsFor_diff,xGoalsPercentage_diff,xOnGoalFor_diff,xPlayContinuedInZoneFor_diff,xPlayContinuedOutsideZoneFor_diff,xPlayStoppedFor_diff,xReboundsFor_diff
0,0,7.0,0.1694,-3.0,-7.0,0.1494,1.847,1.636,8.0,-3.0,...,1.319,1.926,0.296,0.156,0.5050,7.602,5.623,3.086,0.301,0.745
1,0,4.0,0.0000,2.0,-6.0,-0.0512,-0.831,-1.031,3.0,4.0,...,-0.730,-0.825,-0.987,-0.051,-0.1572,-3.222,-0.983,-1.367,-0.065,-0.031
2,0,11.0,0.1526,-1.0,3.0,0.0770,0.746,0.330,1.0,-2.0,...,0.215,1.390,-0.260,0.302,0.1628,5.324,2.844,1.454,0.179,0.918
3,0,-17.0,-0.2942,1.0,-9.0,-0.2000,0.840,0.813,-1.0,2.0,...,-4.413,0.883,0.374,-0.114,0.2156,-12.545,-7.587,-5.952,-0.381,-0.553
4,1,-2.0,-0.0350,1.0,12.0,-0.0250,0.210,0.215,-2.0,0.0,...,0.281,0.257,0.060,-0.074,0.0758,0.001,-1.432,-0.704,-0.046,-0.357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0,7.0,0.0240,1.0,6.0,-0.0416,-0.557,-0.797,-3.0,-5.0,...,-0.121,-0.578,-0.196,-0.014,-0.1058,-4.593,-0.513,-2.720,-0.002,-0.066
205,1,-2.0,-0.1380,-2.0,-2.0,-0.1556,-0.502,-0.610,-4.0,-5.0,...,-2.543,-0.478,0.213,-0.183,-0.0994,-9.825,-6.283,-3.729,-0.338,-0.628
206,1,0.0,-0.2056,0.0,-10.0,-0.2816,-1.596,-1.683,-6.0,0.0,...,-5.010,-1.672,0.050,-0.385,-0.2642,-21.655,-11.799,-8.213,-0.691,-1.613
207,1,-9.0,-0.0802,4.0,7.0,-0.0200,0.483,0.162,2.0,7.0,...,0.801,0.774,0.958,-0.022,0.1126,-1.569,-2.094,-1.427,-0.005,-0.049


In [2]:
model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)


In [3]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        "learning_rate": [0.1, 0.2, 0.3],
        "n_estimators": [200, 400, 600],
        
        }

In [4]:
df_diff_target = pd.read_csv('train-set-1.csv')
X = df_diff_target.iloc[:,1:]
y = df_diff_target.iloc[:,0]

In [5]:
folds = 5
param_comb = 7

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(model, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=4, cv=skf.split(X,y), verbose=3, random_state=1001)

In [6]:
random_search.fit(X,y)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[CV 2/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.960 total time=   0.7s
[CV 1/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.983 total time=   0.7s
[CV 4/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.958 total time=   0.7s
[CV 3/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.958 total time=   0.7s
[CV 5/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.955 total time=   0.7s
[CV 1/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=200, subsample=0.8;, score=0.982 total time=   0.8s
[CV 2/5] END colsample

RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x164bf3230>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=No...
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=7, n_jobs=4,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 5],
                                        'learning_rate': [0.1, 0.2, 0.3],
                                        'max_depth': [3, 4, 5],
                                        'min_child_weight': [1, 5, 10],
                                        'n_estimators': [200, 400, 600],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=1001, scoring='accuracy', verbose=3)

In [265]:
best_model = random_search.best_estimator_

In [266]:
type(best_model)

xgboost.sklearn.XGBClassifier

In [278]:
best_params = best_model.__dict__

In [280]:
model = xgb.XGBClassifier(**best_params)

In [281]:
model.fit(X_train, y_train)

[13:36:39] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_b2k4a2hea3/croot/xgboost-split_1675457783214/work/src/learner.cc:767: 
Parameters: { "_Booster", "classes_", "n_classes_" } are not used.



XGBClassifier(_Booster=<xgboost.core.Booster object at 0x16a6bfe00>,
              base_score=None, booster=None, callbacks=None,
              classes_=array([0, 1]), colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.6,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.5, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_classes_=2, n_estimators=200, n_jobs=None, ...)

In [282]:
model.save_model('model')

/Users/kilianhammersmith/Documents/GitHub/NHL-Win-Predictor/.conda/lib/python3.12/site-packages/xgboost/sklearn.py:761: UserWarning: kwargs is not saved in Scikit-Learn meta.
  warnings.warn(


In [273]:
X.iloc[0,:].to_frame().transpose()

,blockedShotAttemptsFor_diff,corsiPercentage_diff,dZoneGiveawaysFor_diff,faceOffsWonFor_diff,fenwickPercentage_diff,flurryAdjustedxGoalsFor_diff,flurryScoreVenueAdjustedxGoalsFor_diff,freezeFor_diff,giveawaysFor_diff,goalsAgainst_diff,...,xFreezeFor_diff,xGoalsFor_diff,xGoalsFromActualReboundsOfShotsFor_diff,xGoalsFromxReboundsOfShotsFor_diff,xGoalsPercentage_diff,xOnGoalFor_diff,xPlayContinuedInZoneFor_diff,xPlayContinuedOutsideZoneFor_diff,xPlayStoppedFor_diff,xReboundsFor_diff
0,7.0,0.1694,-3.0,-7.0,0.1494,1.847,1.636,8.0,-3.0,3.0,...,1.319,1.926,0.296,0.156,0.505,7.602,5.623,3.086,0.301,0.745


In [220]:
corrs = json.load(open('correlations.json', 'r'))

In [224]:
for key, val in corrs.items():
    print(key, val)

target 0.9999999999999999
blockedShotAttemptsFor_diff 0.17756914864731607
corsiPercentage_diff 0.03029981174674833
dZoneGiveawaysFor_diff 0.025399091247991515
faceOffsWonFor_diff 0.07350697250495433
fenwickPercentage_diff 0.05203597460010471
flurryAdjustedxGoalsFor_diff 0.40745785220070785
flurryScoreVenueAdjustedxGoalsFor_diff 0.46260774232542046
freezeFor_diff 0.10045343882577956
giveawaysFor_diff 0.049390407005840645
goalsAgainst_diff 0.8394083377049366
highDangerGoalsFor_diff 0.5456682850532285
highDangerShotsFor_diff 0.2952638085463261
highDangerxGoalsFor_diff 0.39969858646307554
hitsFor_diff 0.14724772279996803
lowDangerGoalsFor_diff 0.45181708395812503
lowDangerShotsFor_diff 0.0847984842628787
lowDangerxGoalsFor_diff 0.002283044892423019
mediumDangerGoalsFor_diff 0.497087084527205
mediumDangerShotsFor_diff 0.15908553126536673
mediumDangerxGoalsFor_diff 0.16228220249456332
missedShotsFor_diff 0.07589696863071493
penalityMinutesFor_diff 0.013780631140319726
penaltiesFor_diff 0.002

In [225]:
high_corrs = [key for key, value in corrs.items() if value >= 0.1]

In [260]:
df_diff_target_high = df_diff_target[high_corrs]

In [261]:
df_diff_target_high

,target,blockedShotAttemptsFor_diff,flurryAdjustedxGoalsFor_diff,flurryScoreVenueAdjustedxGoalsFor_diff,freezeFor_diff,goalsAgainst_diff,highDangerGoalsFor_diff,highDangerShotsFor_diff,highDangerxGoalsFor_diff,hitsFor_diff,...,scoreAdjustedShotsAttemptsFor_diff,scoreAdjustedTotalShotCreditFor_diff,scoreAdjustedUnblockedShotAttemptsFor_diff,scoreFlurryAdjustedTotalShotCreditFor_diff,scoreVenueAdjustedxGoalsFor_diff,shotsOnGoalFor_diff,takeawaysFor_diff,totalShotCreditFor_diff,xGoalsFor_diff,xGoalsPercentage_diff
0,0,7.0,1.847,1.636,8.0,3.0,0.0,3.0,0.803,-10.0,...,11.495,1.526,7.600,1.497,1.715,11.0,-3.0,1.786,1.926,0.5050
1,0,4.0,-0.831,-1.031,3.0,3.0,-2.0,-3.0,-1.973,6.0,...,-5.949,-0.126,-7.876,-0.149,-1.029,5.0,-1.0,0.111,-0.825,-0.1572
2,0,11.0,0.746,0.330,1.0,3.0,-4.0,0.0,-0.217,18.0,...,9.750,1.664,1.326,1.210,0.953,-2.0,-3.0,1.953,1.390,0.1628
3,0,-17.0,0.840,0.813,-1.0,1.0,0.0,2.0,0.592,-4.0,...,-32.586,0.371,-16.183,0.381,0.855,-11.0,-3.0,0.396,0.883,0.2156
4,1,-2.0,0.210,0.215,-2.0,-4.0,1.0,0.0,0.070,-3.0,...,0.033,0.121,-0.735,0.109,0.262,1.0,0.0,0.123,0.257,0.0758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0,7.0,-0.557,-0.797,-3.0,0.0,0.0,-1.0,-0.431,-15.0,...,-1.529,-0.651,-7.341,-0.654,-0.823,-2.0,1.0,-0.395,-0.578,-0.1058
205,1,-2.0,-0.502,-0.610,-4.0,0.0,0.0,0.0,-0.347,-9.0,...,-17.632,-1.007,-14.898,-0.991,-0.589,-9.0,0.0,-0.874,-0.478,-0.0994
206,1,0.0,-1.596,-1.683,-6.0,0.0,-1.0,-1.0,-0.555,-1.0,...,-29.659,-2.202,-29.156,-2.125,-1.762,-24.0,-1.0,-2.107,-1.672,-0.2642
207,1,-9.0,0.483,0.162,2.0,0.0,1.0,3.0,0.925,5.0,...,-18.067,-0.527,-6.981,-0.584,0.450,0.0,2.0,-0.207,0.774,0.1126


In [262]:
X_high = df_diff_target_high.iloc[:, 1:]
y_high = df_diff_target.iloc[:, 0]

In [263]:
model_high = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)
folds = 5
param_comb = 7

random_search_high = RandomizedSearchCV(model_high, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=None, cv=skf.split(X_high,y_high), verbose=3, random_state=1001)

In [264]:
random_search_high.fit(X_high, y_high)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV 1/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.980 total time=   0.3s
[CV 2/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.963 total time=   0.3s
[CV 3/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.958 total time=   0.3s
[CV 4/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.955 total time=   0.3s
[CV 5/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=200, subsample=0.6;, score=0.948 total time=   0.3s
[CV 1/5] END colsample_bytree=1.0, gamma=1.5, learning_rate=0.1, max_depth=4, min_child_weight=5, n_estimators=200, subsample

RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x16ccd9cf0>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=No...
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=7,
                   param_distributions={'colsample_bytree': [0.6, 0.8, 1.0],
                                        'gamma': [0.5, 1, 1.5, 2, 5],
                                        'learning_rate': [0.1, 0.2, 0.3],
                                        'max_depth': [3, 4, 5],
                                        'min_child_weight': [1, 5, 10],
                                        'n_estimators': [200, 400, 600],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=1001, scoring='accuracy', verbose=3)